In [ ]:
# Install required packages (if needed)
# !pip install dspy-ai mlflow python-dotenv

import dspy
import mlflow
import os
from dotenv import load_dotenv
from typing import List, Optional
from datetime import datetime

print("✅ All imports successful!")

✅ All imports successful!


In [2]:
# Load environment variables
load_dotenv()

# Set up MLflow experiment
mlflow.set_experiment("DSPy-Airline-Agent-Gemini")

# Enable DSPy auto-tracing (ONE LINE - as per transcript)
mlflow.dspy.autolog()

print("🔍 MLflow tracing enabled!")
print(f"📊 Experiment: DSPy-Airline-Agent-Gemini")

2026/02/04 12:30:51 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.schemas
2026/02/04 12:30:51 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.tables
2026/02/04 12:30:51 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.types
2026/02/04 12:30:51 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.constraints
2026/02/04 12:30:51 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.defaults
2026/02/04 12:30:51 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.comments
2026/02/04 12:30:51 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2026/02/04 12:30:51 INFO mlflow.store.db.utils: Updating database tables
2026/02/04 12:30:51 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/02/04 12:30:51 INFO alembic.runtime.migration: Will assume non-transactional DDL.
2026/02/04 12:30:52 INFO alembic.runtime.migration: Running upgrade  -> 451aebb31d03, add metric step
2026/02/04 12:3

🔍 MLflow tracing enabled!
📊 Experiment: DSPy-Airline-Agent-Gemini


In [3]:
# Configure Gemini 2.5 Flash
lm = dspy.LM(
    'gemini/gemini-2.5-flash',  # Updated model
    api_key=os.getenv('GEMINI_API_KEY'),
    max_tokens=2000,
    temperature=0.7
)

# Set as default LM
dspy.configure(lm=lm)

print("🚀 Gemini 2.5 Flash configured!")
print(f"Model: {lm.model}")

# Quick connection test
test = dspy.Predict("question -> answer")
result = test(question="What is 5 + 3?")
print(f"Test: 5 + 3 = {result.answer}")
print("✅ DSPy + Gemini 2.5 Flash is ready!")


🚀 Gemini 2.5 Flash configured!
Model: gemini/gemini-2.5-flash
Test: 5 + 3 = 8
✅ DSPy + Gemini 2.5 Flash is ready!


d:\User\Desktop\Acads\4th Year\TwoTabs Dir\.venv\Lib\site-packages\pydantic\main.py:528: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 7: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='[[ ## an...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...r_specific_fields=None)), input_type=Choices])
  return self.__pydantic_serializer__.to_json(


In [4]:
# Mock database structures (as per transcript)
USER_DB = {
    "john_doe": {
        "name": "John Doe",
        "email": "john@example.com",
        "frequent_flyer_number": "FF12345"
    }
}

FLIGHTS_DB = {
    "2026-02-10": [
        {"flight_id": "AA101", "origin": "SFO", "destination": "JFK", 
         "departure": "08:00", "arrival": "16:30", "duration": "5.5h", "price": 350},
        {"flight_id": "UA202", "origin": "SFO", "destination": "JFK", 
         "departure": "14:00", "arrival": "22:00", "duration": "5.0h", "price": 420}
    ]
}

ITINERARY_DB = []  # Stores booked flights
TICKETS_DB = []    # Stores support tickets

print("📦 Mock database initialized")
print(f"   - Users: {len(USER_DB)}")
print(f"   - Flight routes: {len(FLIGHTS_DB)}")


📦 Mock database initialized
   - Users: 1
   - Flight routes: 1


In [5]:
def search_flights(date: str, origin: str, destination: str) -> List[dict]:
    """
    Search for available flights on a specific date between two cities.
    
    Args:
        date: Flight date in YYYY-MM-DD format
        origin: Three-letter airport code (e.g., SFO)
        destination: Three-letter airport code (e.g., JFK)
    
    Returns:
        List of available flights with details
    """
    flights = FLIGHTS_DB.get(date, [])
    return [f for f in flights if f["origin"] == origin and f["destination"] == destination]


def pick_best_flight(flight_candidates: List[dict]) -> dict:
    """
    Pick the best flight from candidates based on duration and price.
    Prefers shortest duration, then lowest price.
    
    Args:
        flight_candidates: List of flight dictionaries
    
    Returns:
        Best flight option
    """
    if not flight_candidates:
        return {}
    
    # Sort by duration first, then price
    sorted_flights = sorted(flight_candidates, 
                           key=lambda x: (float(x["duration"].replace("h", "")), x["price"]))
    return sorted_flights[0]


def get_user_info(username: str) -> dict:
    """
    Retrieve user profile information from the database.
    
    Args:
        username: User's username identifier
    
    Returns:
        User profile dictionary
    """
    return USER_DB.get(username, {})


def book_flight(flight_id: str, user_email: str, username: str) -> dict:
    """
    Book a flight for a user and generate a confirmation number.
    
    Args:
        flight_id: Unique flight identifier (e.g., AA101)
        user_email: User's email address
        username: User's username
    
    Returns:
        Booking confirmation with confirmation number
    """
    confirmation_number = f"CONF-{len(ITINERARY_DB) + 1001}"
    booking = {
        "confirmation_number": confirmation_number,
        "flight_id": flight_id,
        "user_email": user_email,
        "username": username,
        "booked_at": datetime.now().isoformat()
    }
    ITINERARY_DB.append(booking)
    return booking


def cancel_itinerary(confirmation_number: str) -> dict:
    """
    Cancel an existing flight booking.
    
    Args:
        confirmation_number: Booking confirmation number
    
    Returns:
        Cancellation status
    """
    for booking in ITINERARY_DB:
        if booking["confirmation_number"] == confirmation_number:
            ITINERARY_DB.remove(booking)
            return {"status": "cancelled", "confirmation": confirmation_number}
    return {"status": "not_found"}


def create_support_ticket(issue_description: str, user_email: str) -> dict:
    """
    Create a customer support ticket when automatic resolution fails.
    
    Args:
        issue_description: Description of the customer's issue
        user_email: Customer's email address
    
    Returns:
        Support ticket with ticket number
    """
    ticket_number = f"TICKET-{len(TICKETS_DB) + 5001}"
    ticket = {
        "ticket_number": ticket_number,
        "issue": issue_description,
        "user_email": user_email,
        "created_at": datetime.now().isoformat()
    }
    TICKETS_DB.append(ticket)
    return ticket


print("🛠️ Tools defined:")
print("   - search_flights")
print("   - pick_best_flight")
print("   - get_user_info")
print("   - book_flight")
print("   - cancel_itinerary")
print("   - create_support_ticket")


🛠️ Tools defined:
   - search_flights
   - pick_best_flight
   - get_user_info
   - book_flight
   - cancel_itinerary
   - create_support_ticket


In [6]:
class AirlineAgentSignature(dspy.Signature):
    """
    You are an airline customer service agent. Help users book flights,
    modify itineraries, and resolve issues. Be helpful and professional.
    """
    user_request: str = dspy.InputField(desc="The customer's request or question")
    process_result: str = dspy.OutputField(desc="Final response with confirmation numbers or ticket IDs")


print("✍️ Signature defined: AirlineAgentSignature")


✍️ Signature defined: AirlineAgentSignature


In [7]:
# Combine signature and tools into ReAct module
agent = dspy.ReAct(
    signature=AirlineAgentSignature,
    tools=[
        search_flights,
        pick_best_flight,
        get_user_info,
        book_flight,
        cancel_itinerary,
        create_support_ticket
    ],
    max_iters=10  # Maximum reasoning steps
)

print("🤖 ReAct agent created with 6 tools")
print(f"   Max iterations: 10")


🤖 ReAct agent created with 6 tools
   Max iterations: 10


In [8]:
# Test case: Book a flight
user_query = """
I need to book a flight from SFO to JFK on February 10, 2026. 
My name is john_doe and I prefer the shortest flight.
"""

print("📝 User Request:")
print(user_query)
print("\n" + "="*60 + "\n")

# Invoke the agent (MLflow will auto-trace this)
result = agent(user_request=user_query)

print("✅ Agent Response:")
print(result.process_result)


📝 User Request:

I need to book a flight from SFO to JFK on February 10, 2026. 
My name is john_doe and I prefer the shortest flight.





d:\User\Desktop\Acads\4th Year\TwoTabs Dir\.venv\Lib\site-packages\pydantic\main.py:528: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 7: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='[[ ## ne...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...r_specific_fields=None)), input_type=Choices])
  return self.__pydantic_serializer__.to_json(
d:\User\Desktop\Acads\4th Year\TwoTabs Dir\.venv\Lib\site-packages\pydantic\main.py:528: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 7: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='[[ ## ne...er_specific_fields=None), input_type=

✅ Agent Response:
Your flight UA202 from SFO to JFK on February 10, 2026, departing at 14:00 and arriving at 22:00, has been successfully booked for John Doe. Your confirmation number is CONF-1001. A confirmation email has been sent to john@example.com.


d:\User\Desktop\Acads\4th Year\TwoTabs Dir\.venv\Lib\site-packages\pydantic\main.py:528: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 7: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='[[ ## re...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...r_specific_fields=None)), input_type=Choices])
  return self.__pydantic_serializer__.to_json(


In [17]:
import pandas as pd

# Simple check: did the tracing work?
try:
    traces = mlflow.search_traces(max_results=1)
    
    if not traces.empty:
        print("✅ MLflow tracing is working!")
        print(f"\n📊 Captured {len(traces)} trace(s)")
        print(f"\nColumns available: {list(traces.columns)}")
        
        # Show the first trace
        print("\n🔍 First Trace Preview:")
        first_trace = traces.iloc[0]
        print(first_trace.to_dict())
        
    else:
        print("⚠️ No traces found. The agent may not have run yet.")
        
except Exception as e:
    print(f"❌ Error accessing traces: {e}")
    print("\nMake sure:")
    print("  1. mlflow.dspy.autolog() was called")
    print("  2. The agent has been invoked at least once")


✅ MLflow tracing is working!

📊 Captured 1 trace(s)

Columns available: ['trace_id', 'trace', 'client_request_id', 'state', 'request_time', 'execution_duration', 'request', 'response', 'trace_metadata', 'tags', 'spans', 'assessments']

🔍 First Trace Preview:
{'trace_id': 'tr-42ac91fee072ecab80ee00b47d74ad8b', 'trace': '{"info": {"trace_id": "tr-42ac91fee072ecab80ee00b47d74ad8b", "trace_location": {"type": "MLFLOW_EXPERIMENT", "mlflow_experiment": {"experiment_id": "1"}}, "request_time": "2026-02-04T04:31:52.008Z", "state": "OK", "trace_metadata": {"mlflow.trace_schema.version": "3", "mlflow.traceInputs": "{\\"user_request\\": \\"\\\\nI need to book a flight from SFO to JFK on February 10, 2026. \\\\nMy name is john_doe and I prefer the shortest flight.\\\\n\\"}", "mlflow.trace.sizeStats": "{\\"total_size_bytes\\": 178280, \\"num_spans\\": 30, \\"max\\": 15192, \\"p25\\": 2810, \\"p50\\": 5274, \\"p75\\": 7981}", "mlflow.source.type": "NOTEBOOK", "mlflow.source.name": "lesson_2.ipynb", 

In [18]:
# Test case: Issue that requires human support
edge_case_query = """
I need to change my seat to first class on flight AA101 
but the system won't let me upgrade.
"""

print("📝 Edge Case Request:")
print(edge_case_query)
print("\n" + "="*60 + "\n")

result_edge = agent(user_request=edge_case_query)

print("✅ Agent Response:")
print(result_edge.process_result)

# Check if ticket was created
if TICKETS_DB:
    print(f"\n🎫 Support ticket created: {TICKETS_DB[-1]['ticket_number']}")


📝 Edge Case Request:

I need to change my seat to first class on flight AA101 
but the system won't let me upgrade.





d:\User\Desktop\Acads\4th Year\TwoTabs Dir\.venv\Lib\site-packages\pydantic\main.py:528: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 7: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content="[[ ## ne...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...r_specific_fields=None)), input_type=Choices])
  return self.__pydantic_serializer__.to_json(
d:\User\Desktop\Acads\4th Year\TwoTabs Dir\.venv\Lib\site-packages\pydantic\main.py:528: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 7: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='[[ ## ne...er_specific_fields=None), input_type=

✅ Agent Response:
I understand you're having trouble upgrading your seat on flight AA101. I've created a support ticket for you with number TICKET-5001. A support agent will review your request and contact you shortly to assist with the upgrade.

🎫 Support ticket created: TICKET-5001


d:\User\Desktop\Acads\4th Year\TwoTabs Dir\.venv\Lib\site-packages\pydantic\main.py:528: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 7: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content="[[ ## re...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...r_specific_fields=None)), input_type=Choices])
  return self.__pydantic_serializer__.to_json(


In [19]:
print("📊 DATABASE STATE AFTER OPERATIONS:")
print("\n1. ITINERARY_DB (Bookings):")
for booking in ITINERARY_DB:
    print(f"   - {booking['confirmation_number']}: {booking['flight_id']} for {booking['username']}")

print("\n2. TICKETS_DB (Support Tickets):")
for ticket in TICKETS_DB:
    print(f"   - {ticket['ticket_number']}: {ticket['issue'][:50]}...")

print(f"\n✅ Total bookings: {len(ITINERARY_DB)}")
print(f"✅ Total tickets: {len(TICKETS_DB)}")


📊 DATABASE STATE AFTER OPERATIONS:

1. ITINERARY_DB (Bookings):
   - CONF-1001: UA202 for john_doe

2. TICKETS_DB (Support Tickets):
   - TICKET-5001: User wants to change seat to first class on flight...

✅ Total bookings: 1
✅ Total tickets: 1


In [ ]:
# If running MLflow locally, you can launch the UI
print("🌐 To view traces in MLflow UI:")
print("   1. Open terminal")
print("   2. Run: mlflow ui")
print("   3. Navigate to: http://localhost:5000")
print("   4. Select experiment: DSPy-Airline-Agent-Gemini")
print("\n💡 You'll see hierarchical traces showing:")
print("   - ReAct module calls")
print("   - Tool executions (search_flights, book_flight, etc.)")
print("   - LLM prompts and responses")


🌐 To view traces in MLflow UI:
   1. Open terminal
   2. Run: mlflow ui
   3. Navigate to: http://localhost:5000
   4. Select experiment: DSPy-Airline-Agent-Gemini

💡 You'll see hierarchical traces showing:
   - ReAct module calls
   - Tool executions (search_flights, book_flight, etc.)
   - LLM prompts and responses
   - Execution times and errors
